# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [14]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [15]:
env = UnityEnvironment(file_name="Reacher_Linux/Reacher.x86")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 
ERROR:root:Exception calling application: Ran out of input
Traceback (most recent call last):
  File "/home/mohit/anaconda3/envs/drlnd/lib/python3.6/site-packages/grpc/_server.py", line 385, in _call_behavior
    return behavior(argument, context), True
  File "/home/mohit/anaconda3/envs/drlnd/lib/python3.6/site-packages/unityagents/rpc_communicator.py", line 26, in Exchange
    return sel

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [9]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [10]:
print(env.brain_names)

['ReacherBrain']


### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [11]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


#### Testing parallel agents with neural net

In [12]:
from rlcc.network.network_bodies import FCBody
from rlcc.network.network_heads import GaussianActorCriticNet
import importlib
import rlcc
importlib.reload(rlcc.network.network_bodies)
importlib.reload(rlcc.network.network_heads)

ModuleNotFoundError: No module named 'rlcc.network.network_bodies'

In [27]:
import torch.nn.functional as F
net = GaussianActorCriticNet(brain.vector_observation_space_size, 
                             brain.vector_action_space_size, 
                             FCBody(brain.vector_observation_space_size, hidden_units=(64, 64), gate=F.tanh), 
                             FCBody(brain.vector_observation_space_size, hidden_units=(64, 64), gate=F.tanh),
                             )

In [78]:
import torch
y = net(states)
mean = y['mean']
dist = torch.distributions.Normal(mean, F.softplus(torch.nn.Parameter(torch.zeros(brain.vector_action_space_size))))
log_prob = dist.log_prob(y['a']).sum(-1).unsqueeze(-1)
print(dist.log_prob(y['a']))
print(y['log_pi_a'][0], log_prob[0])

tensor([[-1.3609, -1.6885, -0.5533, -0.8695],
        [-1.2738, -1.5589, -1.2387, -0.7405],
        [-2.0234, -0.5619, -0.9738, -5.3231],
        [-0.5525, -0.5893, -0.6365, -0.5881],
        [-0.8992, -1.1821, -0.5690, -0.6655],
        [-1.7399, -1.9414, -0.5623, -0.6603],
        [-0.6395, -2.4075, -0.7476, -0.5579],
        [-0.6051, -0.5817, -0.5556, -0.6926],
        [-0.7716, -0.8851, -0.6985, -1.9888],
        [-1.3714, -2.7694, -0.7056, -0.5626],
        [-1.5762, -0.6998, -0.8156, -1.8136],
        [-1.1081, -1.2967, -0.9983, -0.9245],
        [-0.5567, -0.6189, -0.5585, -1.2003],
        [-0.5720, -1.8515, -0.5566, -2.6750],
        [-0.6327, -0.9389, -0.6529, -0.6472],
        [-0.8943, -1.1382, -0.9434, -0.5564],
        [-1.3527, -1.1207, -0.5623, -0.6833],
        [-0.7114, -0.9283, -0.6595, -1.1866],
        [-0.7115, -0.9788, -1.6343, -0.9281],
        [-0.7130, -0.6981, -0.6532, -1.0980]])
tensor([-4.4723]) tensor([-4.4723])


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [13]:
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
storage = {'action': [], 'state': [], 'reward': [], 'done': []}
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    storage['action'].append(actions)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    storage['reward'].append(rewards)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.07849999824538827


In [35]:
print(np.array(storage['reward']).shape)

(1001, 20)


In [32]:
import torch
rew = storage['reward'][:5]
rew.append([1., 1., 1., 2., 3., 4. ,1., 6., 3., 4., 1., 1., 1., 2., 3., 4. ,1., 6., 3., 4.])
rew = torch.Tensor(rew)
print(rew)
rewcum = rew.cumsum(-2)
print(rewcum)

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  2.,  3.,  4.,  1.,  6.,  3.,  4.,  1.,  1.,
          1.,  2.,  3.,  4.,  1.,  6.,  3.,  4.]])


ValueError: negative step not yet supported

In [79]:
print(env_info.rewards)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


When finished, you can close the environment.

In [8]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```